## 2RPFS Problem (Cmax objective) - Data treatment of result files  - SP = MIP

In [1]:
import pandas as pd
import numpy as np
import os, fnmatch
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
import glob
import os

%matplotlib inline

In [2]:
import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO

### List files in the output folder 

In [3]:
rootfolder = os.getcwd()
file_list = glob.glob(os.path.join(rootfolder, 'output', 'sp_mip', '*.csv'), recursive=False)
file_list

['/public/doutorado_files/2RPFS_Cmax_Budget/output/sp_mip/separation_cmax_wagner_sp-mip_singlecut-false_200x2.csv',
 '/public/doutorado_files/2RPFS_Cmax_Budget/output/sp_mip/separation_cmax_wagner_sp-mip_singlecut-true_150x2.csv',
 '/public/doutorado_files/2RPFS_Cmax_Budget/output/sp_mip/separation_cmax_wagner_sp-mip_singlecut-true_200x2.csv',
 '/public/doutorado_files/2RPFS_Cmax_Budget/output/sp_mip/separation_cmax_wagner_sp-mip_singlecut-true_200x2_alpha-30.csv',
 '/public/doutorado_files/2RPFS_Cmax_Budget/output/sp_mip/separation_cmax_wagner_sp-mip_singlecut-true_200x2_alpha-40.csv',
 '/public/doutorado_files/2RPFS_Cmax_Budget/output/sp_mip/separation_cmax_wagner_sp-mip_singlecut-true_200x2_alpha-50.csv']

### Read all the CSV files 

In [4]:
# Alternative script to treat files with incorrect number of coluns or faulty lines
def alternative_csv_reader(filename, delimiter=',', header=0, names=None):
    with open(filename, 'r') as file:
        lines = file.readlines() 
        count = 1
        line_list = []
        num_columns = 20
        for line in lines:  # Strips the newline character 
            #print("line{}: {}".format(count, line.strip())) 
            nc = len(line.split(','))
            if 'executionId,' in line:
                #num_columns = nc
                print('Detected {0} columns in CSV file.'.format(nc))
            else:
                if 'none,' in line:
                    if nc == num_columns:
                        line_list.append(line)
                    elif nc > num_columns:  # treat strange truncated lines
                        line = line[line.rfind('none,'):]
                        nc = len(line.split(','))
                        if nc == num_columns:
                            print('WARN: truncating line {0}, for having more columns than expected.'.format(count))
                            line_list.append(line)
                        else:
                            print('WARN: Ignoring line {0}, since it has {1} columns, instead of {2}: '.format(count, nc, num_columns), line)
                    else:  # Ignore line
                        print('WARN: Ignoring line {0}: '.format(count), line)    
                elif len(line_list[-1].split(',')) < num_columns:  # current line is a continuation of the previous one
                    line_list[-1] = line_list[-1].replace('\n', '') + line
                    print('*** Treated line {0}: '.format(count), line_list[-1])
                else:  # Ignore line
                    print('WARN: Ignoring line {0}: '.format(count), line)
            count += 1
        # assert all lines have the same number of columns
        count = 1
        for line in line_list:
            nc = len(line.split(','))
            if nc != num_columns:
                print('ERROR: Line {0} has {1} columns, instead of {2}: '.format(count, nc, num_columns), line)
            count += 1
        text_data = StringIO(''.join(line_list))
        #print('line_list: ', str(line_list))
        #print('text_data: ', text_data)
        df = pd.read_csv(text_data, delimiter=delimiter, header=header, names=names)
        return df

### Process all CSV files and append all data to a single dataframe (one per solution method: Wilson, Wagner) 

In [27]:
%%time

df_wilson = pd.DataFrame()
df_wagner = pd.DataFrame()
for filename in file_list:
    print('Processing file ', filename)
    try:
        df_ = pd.read_csv(filename, delimiter=',', header=0, names=['executionId','ub_name','instance_name','alpha','n','m','Gamma1','Gamma2','cmax','permutation','time_spent','time_to_best_sol','mp_total_time','sp_total_time','iterations','num_visited_solutions','num_improvements','is_optimal','validated','gap','lb','cost','cmax_dp', 'cut_count'])
    except:  # try alternative method to read csv lines
        df_ = alternative_csv_reader(filename, delimiter=',', header=0, names=['executionId','ub_name','instance_name','alpha','n','m','Gamma1','Gamma2','cmax','permutation','time_spent','time_to_best_sol','mp_total_time','sp_total_time','iterations','num_visited_solutions','num_improvements','is_optimal','validated','gap','lb','cost','cmax_dp','cut_count'])
    if 'wagner' not in filename:  # Wilson result file
        df_wilson = df_wilson.append(df_.copy())
    else:  # Wagner result file
        df_wagner = df_wagner.append(df_.copy())

Processing file  /public/doutorado_files/2RPFS_Cmax_Budget/output/sp_mip/separation_cmax_wagner_sp-mip_singlecut-false_200x2.csv
Processing file  /public/doutorado_files/2RPFS_Cmax_Budget/output/sp_mip/separation_cmax_wagner_sp-mip_singlecut-true_150x2.csv
Processing file  /public/doutorado_files/2RPFS_Cmax_Budget/output/sp_mip/separation_cmax_wagner_sp-mip_singlecut-true_200x2.csv
Processing file  /public/doutorado_files/2RPFS_Cmax_Budget/output/sp_mip/separation_cmax_wagner_sp-mip_singlecut-true_200x2_alpha-30.csv
Processing file  /public/doutorado_files/2RPFS_Cmax_Budget/output/sp_mip/separation_cmax_wagner_sp-mip_singlecut-true_200x2_alpha-40.csv
Processing file  /public/doutorado_files/2RPFS_Cmax_Budget/output/sp_mip/separation_cmax_wagner_sp-mip_singlecut-true_200x2_alpha-50.csv
CPU times: user 92.9 ms, sys: 11.6 ms, total: 104 ms
Wall time: 181 ms


In [28]:
df_wagner.head(4)

,executionId,ub_name,instance_name,alpha,n,m,Gamma1,Gamma2,cmax,permutation,...,iterations,num_visited_solutions,num_improvements,is_optimal,validated,gap,lb,cost,cmax_dp,cut_count
0,none,mip_separation,RB2001001_200_2_10_cmax_inputs.txt,10,200,2,20,20,6146.4,27 187 1 130 120 158 103 119 42 76 36 35 174 ...,...,9,9,8,true,true,-1.47972e-16,6146.4,6146.4,6146.4,35
1,none,mip_separation,RB2001001_200_2_10_cmax_inputs.txt,10,200,2,40,20,6298.7,110 97 179 20 140 172 3 147 69 121 182 103 17...,...,10,10,9,true,true,-2.88788e-16,6298.7,6298.7,6298.7,43
2,none,mip_separation,RB2001001_200_2_10_cmax_inputs.txt,10,200,2,60,20,6412.9,112 13 158 128 107 154 23 197 94 2 179 48 174...,...,6,6,5,true,true,-4.25468e-16,6412.9,6412.9,6412.9,22
3,none,mip_separation,RB2001001_200_2_10_cmax_inputs.txt,10,200,2,80,20,6497.1,110 189 36 156 75 93 188 120 35 158 59 10 174...,...,6,6,5,true,false,-1.53912e-05,6497.2,6497.2,6497.1,23


In [29]:
df_wagner['instance_name'] = df_wagner['instance_name'].str.strip()

### Remove duplicated header rows from both dataframes 

In [30]:
def find_invalid_values(df):
    all_invalid_values = set()
    for col in df:
        if col not in ['executionId','ub_name','instance_name','permutation','is_optimal','validated']:
            # 'alpha','n','m','cmax','time_spent','time_to_best_sol','iterations','num_visited_solutions','num_improvements','gap','lb','cost','cmax_dp'
            a = pd.to_numeric(df[col], errors='coerce')
            idx = a.isna()
            invalid_values = df.loc[idx][col].unique()
            all_invalid_values.update(invalid_values)
        #elif col in ['is_optimal','validated']
    print('Invalid values:', all_invalid_values)
    return all_invalid_values

In [31]:
find_invalid_values(df_wagner)

Invalid values: {'gamma2', 'sp_total_time', 'time_spent', 'gap', 'cut_count', 'm', 'gamma1', 'alpha', 'cmax', 'mp_total_time', 'num_improvements', 'time_to_best_sol', 'iterations', 'lb', 'cost', 'cmax_dp', 'num_visited_solutions', 'n'}


{'alpha',
 'cmax',
 'cmax_dp',
 'cost',
 'cut_count',
 'gamma1',
 'gamma2',
 'gap',
 'iterations',
 'lb',
 'm',
 'mp_total_time',
 'n',
 'num_improvements',
 'num_visited_solutions',
 'sp_total_time',
 'time_spent',
 'time_to_best_sol'}

In [32]:
def filter_invalid_values(df):
    # IMPORTANT: AVOID FILTERING 'NAN' VALUES
    for invalid_value in find_invalid_values(df):
        if isinstance(invalid_value, str):  # Evita filtrar os nan
            df = df[~(df == invalid_value).any(axis=1)]
    return df

In [33]:
df_wilson = filter_invalid_values(df_wilson)
find_invalid_values(df_wilson)

Invalid values: set()
Invalid values: set()


set()

In [34]:
df_wagner = filter_invalid_values(df_wagner)
find_invalid_values(df_wagner)

Invalid values: {'gamma2', 'sp_total_time', 'time_spent', 'gap', 'cut_count', 'm', 'gamma1', 'alpha', 'cmax', 'mp_total_time', 'num_improvements', 'time_to_best_sol', 'iterations', 'lb', 'cost', 'cmax_dp', 'num_visited_solutions', 'n'}
Invalid values: set()


set()

### Convert column types from object 

In [35]:
def convert_column_types(df):
    for col in df:
        if col in ['Gamma1','Gamma2','mp_total_time','sp_total_time','alpha','n','m','cmax','time_spent','time_to_best_sol','iterations','num_visited_solutions','num_improvements','gap','lb','cost','cmax_dp']:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        elif col in ['is_optimal','validated']:
            df[col] = df[col].astype('bool')
    return df

In [36]:
%%time
df_wilson = convert_column_types(df_wilson)
df_wagner = convert_column_types(df_wagner)
df_wagner.dtypes

CPU times: user 18.3 ms, sys: 3.78 ms, total: 22 ms
Wall time: 22.1 ms


executionId               object
ub_name                   object
instance_name             object
alpha                      int64
n                          int64
m                          int64
Gamma1                     int64
Gamma2                     int64
cmax                     float64
permutation               object
time_spent               float64
time_to_best_sol         float64
mp_total_time            float64
sp_total_time            float64
iterations                 int64
num_visited_solutions      int64
num_improvements           int64
is_optimal                  bool
validated                   bool
gap                      float64
lb                       float64
cost                     float64
cmax_dp                  float64
cut_count                 object
dtype: object

### Trim existing string columns 

In [37]:
def trim_all_columns(df):
    """
    Trim whitespace from ends of each value across all series in dataframe
    """
    trim_strings = lambda x: x.strip() if isinstance(x, str) else x
    return df.applymap(trim_strings)

In [38]:
df_wilson = trim_all_columns(df_wilson)
df_wagner = trim_all_columns(df_wagner)

### Include a column with the name of the underlying C&CG MILP Model

In [39]:
df_wilson['model'] = 'Wilson'
df_wagner['model'] = 'Wagner'

### Concatenate dataframes 

In [40]:
df = df_wilson.append(df_wagner)
df.head(4)

,model,executionId,ub_name,instance_name,alpha,n,m,Gamma1,Gamma2,cmax,...,iterations,num_visited_solutions,num_improvements,is_optimal,validated,gap,lb,cost,cmax_dp,cut_count
0,Wagner,none,mip_separation,RB2001001_200_2_10_cmax_inputs.txt,10.0,200.0,2.0,20.0,20.0,6146.4,...,9.0,9.0,8.0,True,True,-1.479719e-16,6146.4,6146.4,6146.4,35
1,Wagner,none,mip_separation,RB2001001_200_2_10_cmax_inputs.txt,10.0,200.0,2.0,40.0,20.0,6298.7,...,10.0,10.0,9.0,True,True,-2.887881e-16,6298.7,6298.7,6298.7,43
2,Wagner,none,mip_separation,RB2001001_200_2_10_cmax_inputs.txt,10.0,200.0,2.0,60.0,20.0,6412.9,...,6.0,6.0,5.0,True,True,-4.254681e-16,6412.9,6412.9,6412.9,22
3,Wagner,none,mip_separation,RB2001001_200_2_10_cmax_inputs.txt,10.0,200.0,2.0,80.0,20.0,6497.1,...,6.0,6.0,5.0,True,True,-1.539125e-05,6497.2,6497.2,6497.1,23


In [41]:
df = df.replace(np.nan, -100, regex=True)
df["Gamma1"] = df["Gamma1"].astype(int)
df["Gamma2"] = df["Gamma2"].astype(int)
df["n"] = df["n"].astype(int)
df["m"] = df["m"].astype(int)
df["alpha"] = df["alpha"].astype(int)
df.head(4)

,model,executionId,ub_name,instance_name,alpha,n,m,Gamma1,Gamma2,cmax,...,iterations,num_visited_solutions,num_improvements,is_optimal,validated,gap,lb,cost,cmax_dp,cut_count
0,Wagner,none,mip_separation,RB2001001_200_2_10_cmax_inputs.txt,10,200,2,20,20,6146.4,...,9.0,9.0,8.0,True,True,-1.479719e-16,6146.4,6146.4,6146.4,35
1,Wagner,none,mip_separation,RB2001001_200_2_10_cmax_inputs.txt,10,200,2,40,20,6298.7,...,10.0,10.0,9.0,True,True,-2.887881e-16,6298.7,6298.7,6298.7,43
2,Wagner,none,mip_separation,RB2001001_200_2_10_cmax_inputs.txt,10,200,2,60,20,6412.9,...,6.0,6.0,5.0,True,True,-4.254681e-16,6412.9,6412.9,6412.9,22
3,Wagner,none,mip_separation,RB2001001_200_2_10_cmax_inputs.txt,10,200,2,80,20,6497.1,...,6.0,6.0,5.0,True,True,-1.539125e-05,6497.2,6497.2,6497.1,23


### Fix instance names 

The original instance names, as provided by Ying in the instance file zip, were assembled incorrectly.

The problem lies in the alpha percentage. We are now going to fix this issue.

In [42]:
df['seq'] = df['instance_name'].str[7:9]
df['n_str'] = df['n'].astype(str).str.zfill(3)
df['alpha_str'] = df['alpha'].astype(str)

In [43]:
df['instance_name'] = 'RB' + df['n_str'] + df['alpha_str'] + df['seq'] + '.txt'
df.drop(columns=['n_str', 'alpha_str'], inplace=True)

In [44]:
df.head(4)

,model,executionId,ub_name,instance_name,alpha,n,m,Gamma1,Gamma2,cmax,...,num_visited_solutions,num_improvements,is_optimal,validated,gap,lb,cost,cmax_dp,cut_count,seq
0,Wagner,none,mip_separation,RB2001001.txt,10,200,2,20,20,6146.4,...,9.0,8.0,True,True,-1.479719e-16,6146.4,6146.4,6146.4,35,01
1,Wagner,none,mip_separation,RB2001001.txt,10,200,2,40,20,6298.7,...,10.0,9.0,True,True,-2.887881e-16,6298.7,6298.7,6298.7,43,01
2,Wagner,none,mip_separation,RB2001001.txt,10,200,2,60,20,6412.9,...,6.0,5.0,True,True,-4.254681e-16,6412.9,6412.9,6412.9,22,01
3,Wagner,none,mip_separation,RB2001001.txt,10,200,2,80,20,6497.1,...,6.0,5.0,True,True,-1.539125e-05,6497.2,6497.2,6497.1,23,01


### Round columns containing time (in seconds) 

In [46]:
df['time_spent'] = df['time_spent'].round(2)
df['time_to_best_sol'] = df['time_to_best_sol'].round(2)
df['mp_total_time'] = df['mp_total_time'].round(2)
df['sp_total_time'] = df['sp_total_time'].round(2)

### Sort data according to model, instance_name, alpha, n, m, Gamma1 and Gamma2 and set index

In [47]:
print('Sorting dataset...')
df = df.sort_values(['model', 'n', 'm', 'alpha', 'instance_name', 'Gamma1', 'Gamma2'])
display(df.dtypes)
df = df.set_index(['model', 'n', 'm', 'alpha', 'instance_name', 'Gamma1', 'Gamma2'])
display(df.head(6))

Sorting dataset...


model                     object
executionId               object
ub_name                   object
instance_name             object
alpha                      int64
n                          int64
m                          int64
Gamma1                     int64
Gamma2                     int64
cmax                     float64
permutation               object
time_spent               float64
time_to_best_sol         float64
mp_total_time            float64
sp_total_time            float64
iterations               float64
num_visited_solutions    float64
num_improvements         float64
is_optimal                  bool
validated                   bool
gap                      float64
lb                       float64
cost                     float64
cmax_dp                  float64
cut_count                 object
seq                       object
dtype: object

executionId         ub_name  \
model  n   m alpha instance_name Gamma1 Gamma2                               
Wagner 150 2 10    RB1501001.txt 20     20            none  mip_separation   
                                        40            none  mip_separation   
                                        60            none  mip_separation   
                                        80            none  mip_separation   
                                        100           none  mip_separation   
                                 40     20            none  mip_separation   

                                                  cmax  \
model  n   m alpha instance_name Gamma1 Gamma2           
Wagner 150 2 10    RB1501001.txt 20     20      4501.5   
                                        40      4610.5   
                                        60      4699.9   
                                        80      4766.0   
                                        100     4807.0   
                                 40     20      4579.7   

                                                                                      permutation  \
model  n   m alpha instance_name Gamma1 Gamma2                                                      
Wagner 150 2 10    RB1501001.txt 20     20      78 91 110 69 32 28 22 12 43 142 23 40 42 128 1...   
                                        40      124 103 26 76 46 141 146 29 43 91 126 37 71 13...   
                                        60      124 2 146 4 129 90 71 76 7 134 91 46 22 135 11...   
                                        80      124 2 146 4 129 90 71 76 7 134 91 46 22 135 11...   
                                        100     124 2 146 4 129 90 71 76 7 134 91 46 22 135 11...   
                                 40     20      83 52 34 24 120 81 132 138 5 136 29 25 4 63 30...   

                                                time_spent  time_to_best_sol  \
model  n   m alpha instance_name Gamma1 Gamma2                                 
Wagner 150 2 10    RB1501001.txt 20     20         8635.79           8635.79   
                                        40         7853.84           7853.84   
                                        60           53.92             53.92   
                                        80           20.16             20.16   
                                        100          17.11             17.11   
                                 40     20          122.02            122.02   

                                                mp_total_time  sp_total_time  \
model  n   m alpha instance_name Gamma1 Gamma2                                 
Wagner 150 2 10    RB1501001.txt 20     20            8451.79         146.84   
                                        40            7756.53          94.26   
                                        60              30.84          21.74   
                                        80              10.03           9.02   
                                        100              8.93           7.26   
                                 40     20              87.10          33.13   

                                                iterations  \
model  n   m alpha instance_name Gamma1 Gamma2               
Wagner 150 2 10    RB1501001.txt 20     20            59.0   
                                        40            45.0   
                                        60            11.0   
                                        80             5.0   
                                        100            5.0   
                                 40     20            13.0   

                                                num_visited_solutions  \
model  n   m alpha instance_name Gamma1 Gamma2                          
Wagner 150 2 10    RB1501001.txt 20     20                       59.0   
                                        40                       45.0   
                                        60                       11.0   
    

### Find missing results, for a given value of alpha, n and m

For a given group of alpha, n, m and budget_Gamma, there should be 10 results.

First we will build a dataframe with the instances list and all required budget values.

In [49]:
data = []
rootfolder = os.getcwd()
jobs_folders = glob.glob(os.path.join(rootfolder, 'instances', 'robust', 'ying', 'data', '*/'), recursive=False)
for job_path in jobs_folders:
    alpha_folders = glob.glob(os.path.join(job_path, '*/'), recursive=False)
    n = job_path[job_path.find('data')+5:job_path.rfind('jobs')]
    #print('n: {}'.format(n))
    for alpha_path in alpha_folders:
        alpha = alpha_path[alpha_path.find('jobs')+5:alpha_path.rfind('%')]
        #print('alpha: {}'.format(alpha))
        instance_paths = glob.glob(os.path.join(alpha_path, '*'), recursive=False)
        for instance_path in instance_paths:
            instance_name = instance_path[instance_path.find('%')+2:]
            #print(instance_name)
            for gamma1 in [20, 40, 60, 80, 100]:
                for gamma2 in [20, 40, 60, 80, 100]:
                    for model in ['Wagner']:
                        data.append([model, instance_name, alpha, n, 2, gamma1, gamma2])
df_instances = pd.DataFrame(data, columns=['model', 'instance_name', 'alpha', 'n', 'm', 'Gamma1', 'Gamma2'])
for col in df_instances:
    if col in ['alpha','n','m','Gamma1','Gamma2']:
        df_instances[col] = pd.to_numeric(df_instances[col], errors='coerce')
display(df_instances.dtypes)
df_instances = df_instances.set_index(['model', 'n', 'm', 'alpha', 'instance_name', 'Gamma1', 'Gamma2'])
display(df_instances)

model            object
instance_name    object
alpha             int64
n                 int64
m                 int64
Gamma1            int64
Gamma2            int64
dtype: object

Empty DataFrame
Columns: []
Index: [(Wagner, 100, 2, 10, RB1001001.txt, 20, 20), (Wagner, 100, 2, 10, RB1001001.txt, 20, 40), (Wagner, 100, 2, 10, RB1001001.txt, 20, 60), (Wagner, 100, 2, 10, RB1001001.txt, 20, 80), (Wagner, 100, 2, 10, RB1001001.txt, 20, 100), (Wagner, 100, 2, 10, RB1001001.txt, 40, 20), (Wagner, 100, 2, 10, RB1001001.txt, 40, 40), (Wagner, 100, 2, 10, RB1001001.txt, 40, 60), (Wagner, 100, 2, 10, RB1001001.txt, 40, 80), (Wagner, 100, 2, 10, RB1001001.txt, 40, 100), (Wagner, 100, 2, 10, RB1001001.txt, 60, 20), (Wagner, 100, 2, 10, RB1001001.txt, 60, 40), (Wagner, 100, 2, 10, RB1001001.txt, 60, 60), (Wagner, 100, 2, 10, RB1001001.txt, 60, 80), (Wagner, 100, 2, 10, RB1001001.txt, 60, 100), (Wagner, 100, 2, 10, RB1001001.txt, 80, 20), (Wagner, 100, 2, 10, RB1001001.txt, 80, 40), (Wagner, 100, 2, 10, RB1001001.txt, 80, 60), (Wagner, 100, 2, 10, RB1001001.txt, 80, 80), (Wagner, 100, 2, 10, RB1001001.txt, 80, 100), (Wagner, 100, 2, 10, RB1001001.txt, 100, 20), (Wagner, 100, 2, 10, RB1001001.txt, 100, 40), (Wagner, 100, 2, 10, RB1001001.txt, 100, 60), (Wagner, 100, 2, 10, RB1001001.txt, 100, 80), (Wagner, 100, 2, 10, RB1001001.txt, 100, 100), (Wagner, 100, 2, 10, RB1001002.txt, 20, 20), (Wagner, 100, 2, 10, RB1001002.txt, 20, 40), (Wagner, 100, 2, 10, RB1001002.txt, 20, 60), (Wagner, 100, 2, 10, RB1001002.txt, 20, 80), (Wagner, 100, 2, 10, RB1001002.txt, 20, 100), (Wagner, 100, 2, 10, RB1001002.txt, 40, 20), (Wagner, 100, 2, 10, RB1001002.txt, 40, 40), (Wagner, 100, 2, 10, RB1001002.txt, 40, 60), (Wagner, 100, 2, 10, RB1001002.txt, 40, 80), (Wagner, 100, 2, 10, RB1001002.txt, 40, 100), (Wagner, 100, 2, 10, RB1001002.txt, 60, 20), (Wagner, 100, 2, 10, RB1001002.txt, 60, 40), (Wagner, 100, 2, 10, RB1001002.txt, 60, 60), (Wagner, 100, 2, 10, RB1001002.txt, 60, 80), (Wagner, 100, 2, 10, RB1001002.txt, 60, 100), (Wagner, 100, 2, 10, RB1001002.txt, 80, 20), (Wagner, 100, 2, 10, RB1001002.txt, 80, 40), (Wagner, 100, 2, 10, RB1001002.txt, 80, 60), (Wagner, 100, 2, 10, RB1001002.txt, 80, 80), (Wagner, 100, 2, 10, RB1001002.txt, 80, 100), (Wagner, 100, 2, 10, RB1001002.txt, 100, 20), (Wagner, 100, 2, 10, RB1001002.txt, 100, 40), (Wagner, 100, 2, 10, RB1001002.txt, 100, 60), (Wagner, 100, 2, 10, RB1001002.txt, 100, 80), (Wagner, 100, 2, 10, RB1001002.txt, 100, 100), (Wagner, 100, 2, 10, RB1001003.txt, 20, 20), (Wagner, 100, 2, 10, RB1001003.txt, 20, 40), (Wagner, 100, 2, 10, RB1001003.txt, 20, 60), (Wagner, 100, 2, 10, RB1001003.txt, 20, 80), (Wagner, 100, 2, 10, RB1001003.txt, 20, 100), (Wagner, 100, 2, 10, RB1001003.txt, 40, 20), (Wagner, 100, 2, 10, RB1001003.txt, 40, 40), (Wagner, 100, 2, 10, RB1001003.txt, 40, 60), (Wagner, 100, 2, 10, RB1001003.txt, 40, 80), (Wagner, 100, 2, 10, RB1001003.txt, 40, 100), (Wagner, 100, 2, 10, RB1001003.txt, 60, 20), (Wagner, 100, 2, 10, RB1001003.txt, 60, 40), (Wagner, 100, 2, 10, RB1001003.txt, 60, 60), (Wagner, 100, 2, 10, RB1001003.txt, 60, 80), (Wagner, 100, 2, 10, RB1001003.txt, 60, 100), (Wagner, 100, 2, 10, RB1001003.txt, 80, 20), (Wagner, 100, 2, 10, RB1001003.txt, 80, 40), (Wagner, 100, 2, 10, RB1001003.txt, 80, 60), (Wagner, 100, 2, 10, RB1001003.txt, 80, 80), (Wagner, 100, 2, 10, RB1001003.txt, 80, 100), (Wagner, 100, 2, 10, RB1001003.txt, 100, 20), (Wagner, 100, 2, 10, RB1001003.txt, 100, 40), (Wagner, 100, 2, 10, RB1001003.txt, 100, 60), (Wagner, 100, 2, 10, RB1001003.txt, 100, 80), (Wagner, 100, 2, 10, RB1001003.txt, 100, 100), (Wagner, 100, 2, 10, RB1001004.txt, 20, 20), (Wagner, 100, 2, 10, RB1001004.txt, 20, 40), (Wagner, 100, 2, 10, RB1001004.txt, 20, 60), (Wagner, 100, 2, 10, RB1001004.txt, 20, 80), (Wagner, 100, 2, 10, RB1001004.txt, 20, 100), (Wagner, 100, 2, 10, RB1001004.txt, 40, 20), (Wagner, 100, 2, 10, RB1001004.txt, 40, 40), (Wagner, 100, 2, 10, RB1001004.txt, 40, 60), (Wagner, 100, 2, 10, RB1001004.txt, 40, 80), (Wagner, 100, 2, 10, RB1001004.txt, 40, 100), (Wagner, 100, 2, 10, RB1001004.txt, 60, 20), (Wagner, 100, 2, 10, RB1001004.txt, 60, 40), (Wagner, 100, 2, 

Now, lets join the instances dataframe with the results one (left join).

In [50]:
df_joined = df_instances.join(df, how='left')
df_joined

executionId ub_name  cmax  \
model  n   m alpha instance_name Gamma1 Gamma2                             
Wagner 10  2 10    RB0101001.txt 20     20             NaN     NaN   NaN   
                                        40             NaN     NaN   NaN   
                                        60             NaN     NaN   NaN   
                                        80             NaN     NaN   NaN   
                                        100            NaN     NaN   NaN   
...                                                    ...     ...   ...   
       200 2 50    RB2005010.txt 100    20             NaN     NaN   NaN   
                                        40             NaN     NaN   NaN   
                                        60             NaN     NaN   NaN   
                                        80             NaN     NaN   NaN   
                                        100            NaN     NaN   NaN   

                                               permutation  time_spent  \
model  n   m alpha instance_name Gamma1 Gamma2                           
Wagner 10  2 10    RB0101001.txt 20     20             NaN         NaN   
                                        40             NaN         NaN   
                                        60             NaN         NaN   
                                        80             NaN         NaN   
                                        100            NaN         NaN   
...                                                    ...         ...   
       200 2 50    RB2005010.txt 100    20             NaN         NaN   
                                        40             NaN         NaN   
                                        60             NaN         NaN   
                                        80             NaN         NaN   
                                        100            NaN         NaN   

                                                time_to_best_sol  \
model  n   m alpha instance_name Gamma1 Gamma2                     
Wagner 10  2 10    RB0101001.txt 20     20                   NaN   
                                        40                   NaN   
                                        60                   NaN   
                                        80                   NaN   
                                        100                  NaN   
...                                                          ...   
       200 2 50    RB2005010.txt 100    20                   NaN   
                                        40                   NaN   
                                        60                   NaN   
                                        80                   NaN   
                                        100                  NaN   

                                                mp_total_time  sp_total_time  \
model  n   m alpha instance_name Gamma1 Gamma2                                 
Wagner 10  2 10    RB0101001.txt 20     20                NaN            NaN   
                                        40                NaN            NaN   
                                        60                NaN            NaN   
                                        80                NaN            NaN   
                                        100               NaN            NaN   
...                                                       ...            ...   
       200 2 50    RB2005010.txt 100    20                NaN            NaN   
                                        40                NaN            NaN   
                                        60                NaN            NaN   
                                        80                NaN            NaN   
                                        100               NaN            NaN   

                                                iterations  \
model  n   m alpha instance_name Gamma1 Gamma2               
Wagner 10  2 10    RB0101001.txt 20     20   

Now we will export to CSV a list with all rows with NaN values (missing experimental results).

In [51]:
missing_df = df_joined[df_joined.isnull().any(axis=1)].reset_index()[['model', 'n', 'm', 'alpha', 'instance_name', 'Gamma1', 'Gamma2']]
outputfolder = os.path.join(os.getcwd(), 'results', 'consolidated')
if not os.path.exists(outputfolder):
    os.makedirs(outputfolder)
print('Saving file on folder: ' + outputfolder)
fname = os.path.join(outputfolder, '2RPFS_Cmax_SP-MIP_missing_results.csv')
missing_df.to_csv(fname, sep=';')
print('Saved: ' + fname)

Saving file on folder: /public/doutorado_files/2RPFS_Cmax_Budget/results/consolidated
Saved: /public/doutorado_files/2RPFS_Cmax_Budget/results/consolidated/2RPFS_Cmax_SP-MIP_missing_results.csv


In [53]:
df_grouped = df.groupby(['alpha', 'n', 'm', 'Gamma1', 'Gamma2']).agg({'executionId' : ['count']}).reset_index()
df_grouped.columns = [ ' '.join(str(i) for i in col) for col in df_grouped.columns]
#df_grouped.reset_index(inplace=True)
df_grouped

,alpha,n,m,Gamma1,Gamma2,executionId count
0,10,150,2,20,20,10
1,10,150,2,20,40,10
2,10,150,2,20,60,10
3,10,150,2,20,80,10
4,10,150,2,20,100,10
...,...,...,...,...,...,...
88,50,200,2,40,40,1
89,50,200,2,60,20,1
90,50,200,2,60,40,1
91,50,200,2,80,20,1


In [54]:
table = pd.pivot_table(df, values='executionId', index=['alpha', 'n'], columns=['Gamma1', 'Gamma2'], aggfunc='count', fill_value=0)
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    display(table)

Gamma1    20                  40                  60                  80       \
Gamma2    20  40  60  80  100 20  40  60  80  100 20  40  60  80  100 20  40    
alpha n                                                                         
10    150  10  10  10  10  10  10  10  10  10  10  10  10  10  10  10  10  10   
      200  16  16  14  14  14  16  16  14  14  14  16  14  14  14  14  16  15   
20    150   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   
30    200   1   1   0   0   0   1   1   0   0   0   1   1   0   0   0   1   1   
40    200   3   2   2   2   2   3   2   2   2   2   2   2   2   2   2   2   2   
50    200   1   1   0   0   0   1   1   0   0   0   1   1   0   0   0   1   0   

Gamma1                100                  
Gamma2    60  80  100 20  40  60  80  100  
alpha n                                    
10    150  10  10  10  10  10  10  10  10  
      200  14  15  14  16  14  14  14  14  
20    150   0   0   0   0   0   0   0   0  
30    200   0   0   0   1   0   0   0   0  
40    200   2   2   2   2   2   2   2   2  
50    200   0   0   0   1   0   0   0   0

### Export the dataset to CSV file 

In [55]:
%%time

outputfolder = os.path.join(os.getcwd(), 'results', 'consolidated')
print('Saving file on folder: ' + outputfolder)
fname = os.path.join(outputfolder, '2RPFS_Cmax_SP-MIP_all_results.csv')
df.to_csv(fname, sep=';')
print('Saved: ' + fname)
fname = os.path.join(outputfolder, '2RPFS_Cmax_SP-MIP_all_results.pkl.gz')
df.to_pickle(fname)
print('Saved: ' + fname)

Saving file on folder: /public/doutorado_files/2RPFS_Cmax_Budget/results/consolidated
Saved: /public/doutorado_files/2RPFS_Cmax_Budget/results/consolidated/2RPFS_Cmax_SP-MIP_all_results.csv
Saved: /public/doutorado_files/2RPFS_Cmax_Budget/results/consolidated/2RPFS_Cmax_SP-MIP_all_results.pkl.gz
CPU times: user 234 ms, sys: 3.47 ms, total: 237 ms
Wall time: 337 ms
